In [1]:
# Télécharger les ressources necéssaires pour utiliser le script
import pip
#!pip install python-binance
#!pip install pytz

import os
import pandas as pd
from datetime import date, timedelta
from market_imbalance import MarketImbalance, MarketCandle, create_market_candle, find_imbalances_after_fall, find_imbalances_after_rise
#from analysis_functions import find_imbalances_after_rise, find_imbalance_after_fall
from interact_with_binance import fetch_ohlcv, adjust_timestamps_to_local
from binance.client import Client

from datetime import datetime
import pytz


In [2]:
# Récupérer les clés API à partir des variables d'environnement
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')

# Initialiser le client Binance
client = Client(api_key, api_secret)

server_time = pd.to_datetime(client.get_server_time()['serverTime'], unit='ms')
my_time = datetime.now(pytz.timezone('Europe/Paris'))
print(server_time)
print(my_time)

# Récupérer l'historique des prix pour une paire donnée depuis son listing sur la plateforme
df = fetch_ohlcv(client, symbol = "BTCUSDT", interval=Client.KLINE_INTERVAL_1MINUTE, from_date= "24 Mar, 2024") #"1 Jan, 2015"
df = adjust_timestamps_to_local(df, 'timestamp')

2024-03-24 23:26:12.760000
2024-03-25 00:26:12.925929+01:00


In [6]:
imbalances_after_fall = find_imbalances_after_fall(df)
for imbalance in imbalances_after_fall:
    print(imbalance.timestamp, ",", imbalance.open_price, ",", imbalance.close_price, ",", imbalance.delta_to_be_filled_in)
    

2024-03-24 01:14:00+01:00 , 64326.67000000 , 64271.72000000 , 36.549999999995634
2024-03-24 01:15:00+01:00 , 64271.72000000 , 64172.00000000 , 6.459999999999127
2024-03-24 01:37:00+01:00 , 64463.91000000 , 64416.48000000 , 13.569999999999709
2024-03-24 01:46:00+01:00 , 64506.65000000 , 64428.60000000 , 32.16000000000349
2024-03-24 02:02:00+01:00 , 64530.92000000 , 64509.99000000 , 14.0
2024-03-24 02:13:00+01:00 , 64470.00000000 , 64429.99000000 , 40.0
2024-03-24 02:19:00+01:00 , 64410.02000000 , 64386.01000000 , 4.019999999996799
2024-03-24 02:29:00+01:00 , 64662.38000000 , 64630.01000000 , 19.9800000000032
2024-03-24 02:30:00+01:00 , 64630.02000000 , 64601.43000000 , 14.010000000002037
2024-03-24 02:31:00+01:00 , 64601.42000000 , 64570.06000000 , 29.93000000000029
2024-03-24 02:36:00+01:00 , 64553.92000000 , 64528.71000000 , 17.110000000000582
2024-03-24 02:37:00+01:00 , 64528.71000000 , 64470.01000000 , 21.37999999999738
2024-03-24 02:55:00+01:00 , 64530.01000000 , 64496.93000000 , 1

In [4]:
imbalances_after_rise = find_imbalances_after_rise(df)
for imbalance in imbalances_after_rise:
    print(imbalance.timestamp)

2024-03-24 01:02:00+01:00
2024-03-24 01:07:00+01:00
2024-03-24 01:11:00+01:00
2024-03-24 01:12:00+01:00
2024-03-24 01:23:00+01:00
2024-03-24 01:34:00+01:00
2024-03-24 01:42:00+01:00
2024-03-24 01:43:00+01:00
2024-03-24 01:44:00+01:00
2024-03-24 01:48:00+01:00
2024-03-24 01:49:00+01:00
2024-03-24 01:58:00+01:00
2024-03-24 02:09:00+01:00
2024-03-24 02:23:00+01:00
2024-03-24 02:24:00+01:00
2024-03-24 02:25:00+01:00
2024-03-24 02:26:00+01:00
2024-03-24 02:27:00+01:00
2024-03-24 02:51:00+01:00
2024-03-24 03:05:00+01:00
2024-03-24 03:06:00+01:00
2024-03-24 03:08:00+01:00
2024-03-24 03:09:00+01:00
2024-03-24 03:24:00+01:00
2024-03-24 03:34:00+01:00
2024-03-24 03:36:00+01:00
2024-03-24 03:37:00+01:00
2024-03-24 03:38:00+01:00
2024-03-24 03:43:00+01:00
2024-03-24 03:55:00+01:00
2024-03-24 03:59:00+01:00
2024-03-24 04:16:00+01:00
2024-03-24 04:18:00+01:00
2024-03-24 04:37:00+01:00
2024-03-24 04:38:00+01:00
2024-03-24 04:48:00+01:00
2024-03-24 05:03:00+01:00
2024-03-24 05:04:00+01:00
2024-03-24 0

In [5]:
Imbalance_candles = pd.DataFrame([previous_candles, current_candles, next_candles], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
Imbalance_candles

NameError: name 'previous_candles' is not defined

In [ ]:
# Identifier tous les imbalances

all_imbalances = find_imbalances_after_rise(df) + find_imbalance_after_fall(df)
print(len(all_imbalances))

# Return all imbalances
all_imbalances

In [ ]:
# retourne une liste des jours où il y a eu des imbalances
list_of_imbalance_start = []
for imbalance in all_imbalances :
    list_of_imbalance_start.append(imbalance[0])


In [ ]:
# retourne une liste des jours où il y a eu des imbalances qui n'ont pas été comblés par la suite
random_imbalance_after_a_rise = find_imbalances_after_rise(df)

df_exp = pd.DataFrame(random_imbalance_after_a_rise, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
df_exp['timestamp'] = pd.to_datetime(df_exp['timestamp'], unit='ms')
df_exp.set_index('timestamp', inplace=True)

In [ ]:
# un imbalance après une hausse est comblé si :
imbalance_timestamp = timestamp_imb
imbalance_open = n+1[low]
imbalance_close = n-1[high]
imbalance_depth = imbalance_open - imbalance_close
is_imbalance_filled = "function is_imbalance_filled_(imbalance, toutes les occurences dérrière)"

    """ pour chaque bougie après l'imbalance, regarder le prix le plus bas de la bougie. 
        si le prix le plus est égal ou inférieur à imbalance_close,
        alors is_imbalance_full_filled == true
        & imbalance_was_fullfilled_at = bougie[timestamp]
        
        si le prix le plus bas est strictement inférieur à imbalance_open
        is_imbalance_partially_filled == true
        
        
        mettre à jour la caractéristique de l'imbalance 
    """


In [ ]:
# code à modifier afin de pouvoir travailler avec tradingview sur des timeframes en heure et en minutes.
# Je suis limité à 5000 barres dans tradingview. Je dois donc réduire la liste des imbalances à ceux réalisés dans les 5000 derniers jours à partir d'aujourd'hui
# Get today's date
today = date.today()

# Calculate the date 5000 days ago
days_ago = today - timedelta(days=5000)
print(days_ago)

# Filter dates within the last 5000 days
recent_dates = [d for d in list_of_imbalance_start if d >= days_ago]

# Print recent dates
print("Recent dates within the last 5000 days:")
for d in recent_dates:
    print(d)